# Chroma Vector Store

#### Creating a Chroma Index

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import chromadb

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("quickstart")

#### Load documents, build the GPTVectorStoreIndex with ChromaVectorStore

In [ ]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ChromaVectorStore
from IPython.display import Markdown, display

In [ ]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [ ]:
from llama_index.storage.storage_context import StorageContext


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(
    chroma_collection=chroma_collection
)
response = query_engine.query("What did the author do growing up?")

In [ ]:
display(Markdown(f"<b>{response}</b>"))